## Use stanze to break paragragh into sentence and lemmatize each sentence

In [ ]:
!pip install stanza

In [ ]:
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline
doc = nlp("Barack Obama was born in Hawaii.") # run annotation over a sentence

2021-06-24 00:13:39 INFO: Downloading default packages for language: en (English)...


2021-06-24 00:14:57 INFO: Finished downloading models and saved to C:\Users\xiayu\stanza_resources.
2021-06-24 00:14:57 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-06-24 00:14:58 INFO: Use device: gpu
2021-06-24 00:14:58 INFO: Loading: tokenize
2021-06-24 00:15:00 INFO: Loading: pos
2021-06-24 00:15:00 INFO: Loading: lemma
2021-06-24 00:15:01 INFO: Loading: depparse
2021-06-24 00:15:01 INFO: Loading: sentiment
2021-06-24 00:15:01 INFO: Loading: ner
2021-06-24 00:15:02 INFO: Done loading processors!


In [ ]:

import stanza
import pandas as pd
from ast import literal_eval

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize, lemma')


2021-06-24 00:15:14 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

2021-06-24 00:15:14 INFO: Use device: gpu
2021-06-24 00:15:14 INFO: Loading: tokenize
2021-06-24 00:15:14 INFO: Loading: lemma
2021-06-24 00:15:14 INFO: Done loading processors!


In [ ]:
text_2021 = pd.read_excel("10K_2021.xlsx",index_col=0).dropna().reset_index(drop=True)

In [ ]:
text_2021

,path,para_keywords
0,D:\10-K\10-K\21\0000001800\0001104659-21-02575...,"During the COVID-19 pandemic, Abbott has taken..."
1,D:\10-K\10-K\21\0000001961\0001264931-21-00003...,"After all of our operations were spun off,\nou..."
2,D:\10-K\10-K\21\0000002098\0001564590-21-01712...,Successful Response to Pandemic – Commencing i...
3,D:\10-K\10-K\21\0000002186\0001654954-21-00228...,"In May\n2020, we implemented workforce reducti..."
4,D:\10-K\10-K\21\0000002488\0001628280-21-00118...,•The ongoing novel coronavirus (COVID-19) pand...
...,...,...
2468,D:\10-K\10-K\21\0001822791\0001213900-21-01806...,Our\nclean-surfaced nanocrystals exhibit catal...
2469,D:\10-K\10-K\21\0001823878\0001104659-21-04169...,"Live, Location-Based Entertainment. Live,\nl..."
2470,D:\10-K\10-K\21\0001826018\0001213900-21-01031...,We\nbelieve that combining with a blank check ...
2471,D:\10-K\10-K\21\0001826660\0001493152-21-00665...,The\nCOVID-19 pandemic and resulting global di...


In [ ]:
at_least_num_words = 10
# tokenize paragraph to sentences
def sentences_tokenize(paragraph, nlp): 
    doc = nlp(paragraph)
    return doc.sentences

def preprocess_paragraph(p):
    return p.replace("\n", " ")

# split original text into list of paragraphs (paragraphs are separated by "-------...")
def get_paragraphs_list(ps):
    return [p for p in preprocess_paragraph(ps).split("---------------------------") if p != "\n"]

# tokenize sentence and lemma it
def get_all_sentences(paras):
    sentences = []
    nlp = stanza.Pipeline(lang='en', processors='tokenize, lemma')
    
    for idx, ps in enumerate(paras):
        print(f"\r{round((idx + 1) / paras.shape[0] * 100, 4)}%", end="", flush=True )
        for p in get_paragraphs_list(ps):
            sts = sentences_tokenize(p, nlp)

            for s in sts:
                if len(s.words) > at_least_num_words:
                    sentences.append(s)
            
    return sentences

In [ ]:
sentences = get_all_sentences(text_2021["para_keywords"])

2021-06-24 00:15:23 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

2021-06-24 00:15:23 INFO: Use device: gpu
2021-06-24 00:15:23 INFO: Loading: tokenize
2021-06-24 00:15:23 INFO: Loading: lemma
2021-06-24 00:15:23 INFO: Done loading processors!


100.0%6%

In [ ]:
lemma_sentences = []

for sentence in sentences:
    tmp_sentence = []
    for word in sentence.words:
        tmp_sentence.append(word.lemma)
        
    lemma_sentences.append(tmp_sentence)

# Train word2vec model

In [ ]:
!pip install gensim
from gensim.models import Word2Vec
import numpy as np

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.22
    Uninstalling Cython-0.29.22:
      Successfully uninstalled Cython-0.29.22


C:\Users\xiayu\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
model = Word2Vec(lemma_sentences, min_count=1, vector_size=100, window=5)

In [ ]:
def transform_sentences_to_vector(sentences, model):
    
    vectors = []
    vector_size = model.vector_size
    
    for s in sentences:
        v = np.zeros(vector_size)
        for w in s:
            v += model.wv[w]
        vectors.append(v / len(s))
    
    return vectors
    

In [ ]:
data = transform_sentences_to_vector(lemma_sentences, model)

# KMean

In [ ]:

from nltk.cluster import KMeansClusterer, cosine_distance

In [ ]:
def group_sentence_by_kmean(lemma_sentences, data, num_clusters=8, repeats=5):
    
    clusterer = KMeansClusterer(num_clusters, cosine_distance, repeats=repeats)

    # samples are assigned to cluster labels 
    # starting from 0
    clusters = clusterer.cluster(data, assign_clusters=True)
    
    results = {i: [] for i in set(clusters)}
    
    for idx, c in enumerate(clusters):
        results[c].append(idx)
    
    return clusterer, results

In [ ]:
clusterer, groups = group_sentence_by_kmean(lemma_sentences, data)

In [ ]:
def distance(p1, p2):
    return cosine_distance(p1, p2)

def top_sentences_centroid(centroids, groups, data, lemma_sentences, topn=20):
    top_sentences = []
    
    for cid, idxs in groups.items():
        pair_dists = []
        
        for idx in idxs:
            pair_dists.append(distance(centroids[cid], data[idx]))

        print(f"Cluster {cid}:")
        print("----------------------------")
        
        for i in np.array(pair_dists).argsort()[-topn:]:
            print(" ".join(lemma_sentences[idxs[i]]))
            print("--------------------------------")
        print()
        


In [ ]:
results = top_sentences_centroid(clusterer.means(), groups, data, lemma_sentences)

Cluster 0:
----------------------------
there be currently no vaccine or therapeutic against this virus .
--------------------------------
while several jurisdictions initially propose permanent price control but then implement temporary price control , other jurisdictions may implement permanent price control in the future .
--------------------------------
we conduct this clinical trials at one clinical trial site .
--------------------------------
we have no further payment obligation under the term loan facility and no amount under the secure term loan facility be available for borrowing .
--------------------------------
we may initiate additional phase 2 clinical trials investigate other indications in addition to we plan phase 2 trial in post-transplant aml patient .
--------------------------------
we may initiate additional phase 2 clinical trials investigate other indications in addition to we plan phase 2 trial in post-transplant aml patient .
-------------------------------

• pursue innovative acquisition target that align with we strategy :
--------------------------------
nyu investigator report that no safety concern be identify by the datum safety monitoring board ( dsmb ) .
--------------------------------
there currently be no fda approve laboratory test available to measure T cell immune response to cov - 2 .
--------------------------------
many publish article describe animal model of ards show tempol cause a decrease in lung inflammation and preserve lung pathology associate with acute and chronic lung injury .
--------------------------------
exovacc promotes expansion of resident memory cd8 T cell in the lung
--------------------------------
we corporate headquarters be locate in San Diego , California near major earthquake faults and fire zone .
--------------------------------
we believe that quantify virus -specific T cell may provide important diagnostic advantage because T cell appear early than antibodies and persist long .
-------------

# GMM

In [ ]:
from sklearn import mixture

def group_sentence_by_GMM(lemma_sentences, data, num_clusters=8, repeats=5):
    
    gmm = mixture.GaussianMixture(num_clusters,
                              covariance_type="spherical", random_state=42)

    # samples are assigned to cluster labels 
    # starting from 0
    clusters = gmm.fit_predict(data)
    
    results = {i: [] for i in set(clusters)}
    
    for idx, c in enumerate(clusters):
        results[c].append(idx)
    
    return gmm, results



def distance(p1, p2):
    return cosine_distance(p1, p2)

def top_sentences_centroid(centroids, groups, data, lemma_sentences, topn=20):
    top_sentences = []
    
    for cid, idxs in groups.items():
        pair_dists = []
        
        for idx in idxs:
            pair_dists.append(distance(centroids[cid], data[idx]))

        print(f"Cluster {cid}:")
        print("----------------------------")
        
        for i in np.array(pair_dists).argsort()[-topn:]:
            print(" ".join(lemma_sentences[idxs[i]]))
            print("--------------------------------")
        print()

In [ ]:
clusterer, groups = group_sentence_by_GMM(lemma_sentences, data)
top_sentences_centroid(clusterer.means_, groups, data, lemma_sentences)

Cluster 0:
----------------------------
Net sale in developed Market outside North America increase 14 percent .
--------------------------------
Net sale in developed Market outside North America decrease 4 percent .
--------------------------------
nobility 's fourth quarter sale show significant improvement from the first three quarter of fiscal year 2020 .
--------------------------------
all other entity for fiscal year beginning after December 15 , 2020 , and interim period within fiscal year beginning after December 15 , 2021 .
--------------------------------
creon maintain market leadership in the pancreatic enzyme market with approximately 80 % total market share .
--------------------------------
residential remodeling be up 2 % in 2020 versus 2019 support by stay - at- home mandates .
--------------------------------
the follow table represent note payable balance at December 31 , 2020 :
--------------------------------
Volume increase in packaging & specialty plastics ( up

Cluster 4:
----------------------------
the company believe the impact business 's long - term prospects remain excellent give the company 's attractive market serve , its industry - lead position and prove growth strategy .
--------------------------------
the extent of the impact will depend on future development , which be highly uncertain and cannot be predict .
--------------------------------
however , the future impact of this outbreak be highly uncertain and cannot be predict .
--------------------------------
we cannot , however , reasonably estimate the duration or extent of that volatility .
--------------------------------
the potential negative demand effect on revenue be also uncertain give the volatile environment , but demand and production level be anticipate to continue to recover .
--------------------------------
this reclassifications have no effect on the report result of operation .
--------------------------------
we do not currently anticipate that the covid - 

# LDA

In [ ]:
sentences = []
for s in lemma_sentences:
    sentences.append(" ".join(s))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def group_sentence_by_lda(sentences, num_clusters=8):
    tf_vectorizer = CountVectorizer(min_df=5, stop_words = "english")
    tf = tf_vectorizer.fit_transform(sentences)
    
    tf_feature_names = tf_vectorizer.get_feature_names()
    
    lda = LatentDirichletAllocation(num_clusters, max_iter=30, verbose=1, evaluate_every=1, n_jobs=1, random_state=0)
    
    doc_distribution = lda.fit_transform(tf)
    clusters = doc_distribution.argsort()[:, -1]
    
    results = {i: [] for i in set(clusters)}
    
    for idx, c in enumerate(clusters):
        results[c].append((idx, doc_distribution[idx][c]))
    

    return results

def print_cluster_topn(results, sentences, cluster_number, topn=20):
    topn_sentences = sorted(results[cluster_number], key=lambda x: -x[1])[:topn]
    print(f"Cluster {cluster_number}:\n-------------")
    for idx, score in topn_sentences:
        print(idx, sentences[idx])
        print("---------------------")

In [ ]:
results = group_sentence_by_lda(sentences)

iteration: 1 of max_iter: 30, perplexity: 1160.6417
iteration: 2 of max_iter: 30, perplexity: 968.0436
iteration: 3 of max_iter: 30, perplexity: 887.9567
iteration: 4 of max_iter: 30, perplexity: 850.4900
iteration: 5 of max_iter: 30, perplexity: 830.1455
iteration: 6 of max_iter: 30, perplexity: 817.6508
iteration: 7 of max_iter: 30, perplexity: 809.1029
iteration: 8 of max_iter: 30, perplexity: 802.3094
iteration: 9 of max_iter: 30, perplexity: 795.8939
iteration: 10 of max_iter: 30, perplexity: 788.9921
iteration: 11 of max_iter: 30, perplexity: 784.3235
iteration: 12 of max_iter: 30, perplexity: 781.5200
iteration: 13 of max_iter: 30, perplexity: 779.5094
iteration: 14 of max_iter: 30, perplexity: 777.9794
iteration: 15 of max_iter: 30, perplexity: 776.7986
iteration: 16 of max_iter: 30, perplexity: 775.8866
iteration: 17 of max_iter: 30, perplexity: 775.1944
iteration: 18 of max_iter: 30, perplexity: 774.6542
iteration: 19 of max_iter: 30, perplexity: 774.2043
iteration: 20 of max

In [ ]:
print_cluster_topn(results, sentences, 3, 4)

Cluster 3:
-------------
29869 the covid - 19 pandemic have result in extend travel and other continue restriction in order to reduce the spread of the disease , include California executive order , San Francisco Bay Area order and several other state and local order across the United State , which , among other thing , direct individual to continue to shelter at they place of residence , direct business and governmental agency to cease non-essential operation at physical location , prohibit certain non-essential gathering , and order cessation of non-essential travel .
---------------------
62098 the covid - 19 pandemic have result in extend travel and other continue restriction in order to reduce the spread of the disease , include a California executive order , San Francisco Bay Area order and several other state and local order across the country , which , among other thing , direct individual to continue to shelter at they place of residence , direct business and governmental agen